# Reto final M3

In [1]:
%pip install -qU openai langchain langchain-openai langchain-chroma datasets tiktoken chromadb python-dotenv --upgrade

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Configuración básica del modelo (sin agentes)
import os, json, re, hashlib, tempfile, time
from datetime import datetime
from typing import List, Dict, Any

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from dotenv import load_dotenv

load_dotenv()  # Carga variables definidas en .env
api_key = os.environ.get("OPENAI_API_KEY")

if api_key:
    print("✅ API Key detectada (oculta)")
else:
    raise RuntimeError("❌ Falta OPENAI_API_KEY en .env")

✅ API Key detectada (oculta)


In [3]:
from langchain_openai import ChatOpenAI

# Initialize LangChain OpenAI client and embeddings
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0
)

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """Eres un desarrollador senior experto en C#, .net FRAMEWORK 4.7.2, bases de datos relacionales y arquitecturas escalables.
    Aplica clean code, seguridad (OWASP), y mantenibilidad.
    Responde con explicaciones claras, código estructurado y comentarios técnicos.
    Usa ejemplos prácticos de validación, hashing, JWT y manejo seguro de datos."""),
    
    ("human", "Implementa el siguiente requerimiento y usa las mejores prácticas de seguridad y OWASP.: {text}")
])

In [5]:
chain = chat_prompt_template | llm

In [6]:
history = []

In [7]:
history: List[Dict[str, str]] = []  # formato: [{"role": "user", "content": "..."}, {"role": "assistant", "..."}]

In [8]:
def ask_with_history(user_message: str) -> str:
    global history
    
    # Agregar mensaje del usuario al historial
    history.append({"role": "user", "content": user_message})
    
    # Construir el contexto completo para {text}
    conversation_context = ""
    for msg in history:
        if msg["role"] == "user":
            conversation_context += f"Usuario: {msg['content']}\n\n"
        elif msg["role"] == "assistant":
            conversation_context += f"Asistente: {msg['content']}\n\n"
    
    # Llamar al modelo con TODO el historial dentro de {text}
    result = chain.invoke({"text": conversation_context})
    assistant_response = result.content
    
    # Guardar respuesta en historial
    history.append({"role": "assistant", "content": assistant_response})
    
    return assistant_response

In [9]:
res1 = ask_with_history(
    """
    Necesito implementar validación de entrada robusta para mi API Web en ASP.NET Framework 4.8 (Web API 2).

    Esta es la estructura:
    /DigitalLibrary
    │
    ├── /Backend
    │   ├── DigitalLibrary.sln                # Solución de Visual Studio
    │   ├── DigitalLibrary.csproj              # Proyecto principal
    │   ├── /Controllers                       # Controladores para manejar las solicitudes
    │   │   ├── AuthController.cs              # Controlador para autenticación
    │   │   ├── BooksController.cs             # Controlador para libros
    │   │   └── LoansController.cs             # Controlador para préstamos
    │   ├── /Models                            # Modelos de datos
    │   │   ├── User.cs                        # Modelo de usuario
    │   │   ├── Book.cs                        # Modelo de libro
    │   │   └── Loan.cs                        # Modelo de préstamo
    │   ├── /Data                              # Acceso a datos
    │   │   ├── ApplicationDbContext.cs       # Contexto de la base de datos
    │   │   └── SeedData.cs                   # Datos iniciales para la base de datos
    │   ├── /Services                          # Servicios para lógica de negocio
    │   │   ├── AuthService.cs                 # Servicio de autenticación
    │   │   ├── BookService.cs                 # Servicio de libros
    │   │   └── LoanService.cs                 # Servicio de préstamos

    Tengo estos endpoints en un ApiController:
    - POST /api/auth/register (recibe: name, email, password)
    - POST /api/auth/login (recibe: email, password)

    Crea:
    1. Modelos con DataAnnotations (System.ComponentModel.DataAnnotations)
    2. ViewModels/DTOs para RegisterRequest y LoginRequest con validaciones:
      - Email válido
      - Password: mínimo 8 caracteres, 1 mayúscula, 1 minúscula, 1 número, 1 especial
      - Name: 2-100 caracteres
    3. ActionFilter personalizado para validar ModelState automáticamente
    4. Manejo de errores que retorne IHttpActionResult con BadRequest

    Usa ASP.NET Web API 2, NO ASP.NET Core. Sin minimal APIs, sin Program.cs.
"""
)

In [10]:
print(res1)

Para implementar la validación de entrada robusta en tu API Web utilizando ASP.NET Framework 4.8 (Web API 2), seguiremos los pasos que has mencionado. A continuación, se presenta una estructura detallada que incluye modelos, DTOs, un ActionFilter personalizado y el manejo de errores.

### 1. Modelos con DataAnnotations

Primero, crearemos los modelos `User`, `RegisterRequest` y `LoginRequest` utilizando DataAnnotations para la validación.

#### User.cs
```csharp
using System.ComponentModel.DataAnnotations;

namespace DigitalLibrary.Models
{
    public class User
    {
        public int Id { get; set; }

        [Required]
        [StringLength(100, MinimumLength = 2)]
        public string Name { get; set; }

        [Required]
        [EmailAddress]
        public string Email { get; set; }

        [Required]
        public string PasswordHash { get; set; }
    }
}
```

#### RegisterRequest.cs
```csharp
using System.ComponentModel.DataAnnotations;

namespace DigitalLibrary.Models
{


In [11]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "users.md"
texto = getattr(res1, "content", None) or str(res1)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\users.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\users.md
📏 Tamaño (bytes): 7155
🧪 Existe?: True


In [12]:
res3 = ask_with_history("""
Necesito implementar hash seguro de contraseñas.

Requerimientos:
1. Crear una clase PasswordHasher usando BCrypt.Net-Next
2. Métodos:
   - string HashPassword(string password)
   - bool VerifyPassword(string password, string hash)
3. NO usar ASP.NET Core Identity 
4. Usar BCrypt con work factor de 12
5. Incluir manejo de errores y validación de null

Proporciona:
- Comando NuGet para instalar BCrypt.Net-Next
- Clase completa lista para usar
- Ejemplo de uso en un ApiController

IMPORTANTE: Debe funcionar en .NET Framework 4.7.2, NO .NET Core.

""")

In [13]:
print(res3)

Para implementar un hashing seguro de contraseñas utilizando `BCrypt.Net-Next` en un proyecto de ASP.NET Framework 4.7.2, seguiremos los pasos que has mencionado. A continuación, se incluye el comando NuGet para instalar la biblioteca, la clase `PasswordHasher` y un ejemplo de uso en un `ApiController`.

### 1. Comando NuGet para instalar BCrypt.Net-Next

Para instalar la biblioteca `BCrypt.Net-Next`, puedes usar el siguiente comando en la consola de NuGet:

```bash
Install-Package BCrypt.Net-Next
```

### 2. Clase PasswordHasher

A continuación, se presenta la implementación de la clase `PasswordHasher` que incluye los métodos para hashear y verificar contraseñas, así como el manejo de errores y la validación de entradas nulas.

#### PasswordHasher.cs
```csharp
using BCrypt.Net;
using System;

namespace DigitalLibrary.Services
{
    public class PasswordHasher
    {
        private const int WorkFactor = 12;

        /// <summary>
        /// Hashea una contraseña utilizando BCrypt.
 

In [15]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "hash.md"
texto = getattr(res3, "content", None) or str(res3)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\hash.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\hash.md
📏 Tamaño (bytes): 6273
🧪 Existe?: True


In [16]:
res4 = ask_with_history("""
    Necesito implementar JWT (JSON Web Tokens) .

    Requisitos:
    1. Usar System.IdentityModel.Tokens.Jwt (compatible con Framework 4.8)
    2. Crear clase JwtTokenService con:
    - string GenerateToken(int userId, string email)
    - ClaimsPrincipal ValidateToken(string token)
    3. Leer configuración desde Web.config:
    - SecretKey
    - Issuer
    - Audience
    - ExpirationMinutes
    4. Implementar DelegatingHandler para autenticación JWT en cada request
    5. Atributo [Authorize] funcionando con JWT

    NO uses middleware de ASP.NET Core. 

    Incluye:
    - Comandos NuGet necesarios
    - Configuración en Web.config
    - Registro en WebApiConfig.cs
    - Ejemplo completo en ApiController
""")
print(res4)

Para implementar JWT (JSON Web Tokens) en tu API Web utilizando ASP.NET Framework 4.8, seguiremos los requisitos que has mencionado. A continuación, se incluye la configuración necesaria, la clase `JwtTokenService`, un `DelegatingHandler` para la autenticación y un ejemplo completo en un `ApiController`.

### 1. Comandos NuGet necesarios

Para utilizar JWT en tu proyecto, necesitas instalar el paquete `System.IdentityModel.Tokens.Jwt`. Puedes hacerlo con el siguiente comando en la consola de NuGet:

```bash
Install-Package System.IdentityModel.Tokens.Jwt
```

### 2. Configuración en Web.config

Agrega las siguientes configuraciones en tu archivo `Web.config` para almacenar la clave secreta, el emisor, el público y el tiempo de expiración del token.

```xml
<configuration>
  <appSettings>
    <add key="SecretKey" value="TuClaveSecretaSuperSegura" />
    <add key="Issuer" value="TuEmisor" />
    <add key="Audience" value="TuAudiencia" />
    <add key="ExpirationMinutes" value="60" />
  <

In [17]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "token.md"
texto = getattr(res4, "content", None) or str(res4)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\token.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\token.md
📏 Tamaño (bytes): 9061
🧪 Existe?: True


In [28]:
res5 = ask_with_history("""
    Necesito implementar rate limiting para prevenir brute force.

    Requisitos:
    1. Limitar intentos de login:
    - 5 intentos por IP cada 15 minutos
    - 3 intentos por email cada 10 minutos
    2. Usar ActionFilter o DelegatingHandler (NO middleware de Core)
    3. Almacenar contadores en MemoryCache (System.Runtime.Caching)
    4. Retornar HttpResponseMessage con status 429
    5. Incluir header Retry-After

    NO uses AspNetCoreRateLimit.
    Usa System.Runtime.Caching.MemoryCache.

    Proporciona:
    - Clase RateLimitAttribute completa
    - Configuración en Web.config
    - Registro en WebApiConfig.cs
    - Manejo de storage en memoria

""")
print(res5)

Para implementar un sistema de **rate limiting** que limite los intentos de inicio de sesión en tu API Web utilizando ASP.NET Framework 4.8, seguiremos los requisitos que has mencionado. A continuación, se presenta la implementación completa, que incluye un `ActionFilter` para manejar el rate limiting, la configuración necesaria en `Web.config`, el registro en `WebApiConfig.cs`, y el manejo de almacenamiento en memoria utilizando `System.Runtime.Caching`.

### 1. Clase RateLimitAttribute

La clase `RateLimitAttribute` se encargará de controlar los intentos de inicio de sesión por dirección IP y por correo electrónico. Utilizaremos `MemoryCache` para almacenar los contadores de intentos.

#### RateLimitAttribute.cs
```csharp
using System;
using System.Net;
using System.Net.Http;
using System.Runtime.Caching;
using System.Threading;
using System.Threading.Tasks;
using System.Web.Http;
using System.Web.Http.Controllers;
using System.Web.Http.Filters;

namespace DigitalLibrary.Filters
{
  

In [29]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "rateLimiting.md"
texto = getattr(res5, "content", None) or str(res5)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\rateLimiting.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\rateLimiting.md
📏 Tamaño (bytes): 8828
🧪 Existe?: True


In [30]:
res6 = ask_with_history("""
 Cual sería el codigo de mi AuthController para q esté utilizando el `RateLimitAttribute` para controlar los intentos de inicio de sesión.
""")
print(res6)

Para utilizar el `RateLimitAttribute` en tu `AuthController` y controlar los intentos de inicio de sesión, simplemente necesitas aplicar el atributo a la acción de inicio de sesión (`Login`). Esto asegurará que cada vez que se intente iniciar sesión, se verifiquen los límites de intentos por dirección IP y por correo electrónico.

A continuación, se muestra el código actualizado de tu `AuthController` con el `RateLimitAttribute` aplicado al método `Login`.

### AuthController.cs (actualizado)

```csharp
using DigitalLibrary.Models;
using DigitalLibrary.Services;
using System.Web.Http;

namespace DigitalLibrary.Controllers
{
    [RoutePrefix("api/auth")]
    public class AuthController : ApiController
    {
        private readonly AuthService _authService;
        private readonly JwtTokenService _jwtTokenService;

        public AuthController()
        {
            _authService = new AuthService();
            _jwtTokenService = new JwtTokenService();
        }

        [HttpPost]
 

In [31]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "correcion.md"
texto = getattr(res6, "content", None) or str(res6)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\correcion.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\correcion.md
📏 Tamaño (bytes): 3654
🧪 Existe?: True


In [32]:
res7 = ask_with_history("""

Necesito que me generes un AuthController completo en C# que:
1. Integre todos estos componentes
2. Tenga endpoints POST /register y POST /login
3. Use inyección de dependencias correctamente
4. Incluya manejo de errores con try-catch y respuestas estructuradas
5. Retorne códigos HTTP apropiados (200, 201, 400, 401, 429, 500)
6. Incluya logging con ILogger
7. Siga principios SOLID y Clean Code
8. Tenga comentarios XML para documentación

Incluye también la configuración necesaria en Program.cs para registrar los servicios.
""")
print(res7)

Para crear un `AuthController` completo que integre todos los componentes mencionados, siga los principios de SOLID y Clean Code, y utilice inyección de dependencias, logging y manejo de errores, aquí tienes una implementación detallada.

### 1. Estructura del Proyecto

Asegúrate de que tu proyecto tenga las siguientes dependencias:

- **Microsoft.Extensions.Logging** para el logging.
- **System.Runtime.Caching** para el almacenamiento en memoria.
- **System.IdentityModel.Tokens.Jwt** para la generación de tokens JWT.

### 2. AuthController

Aquí está la implementación del `AuthController`:

#### AuthController.cs
```csharp
using DigitalLibrary.Models;
using DigitalLibrary.Services;
using Microsoft.Extensions.Logging;
using System;
using System.Net;
using System.Net.Http;
using System.Web.Http;

namespace DigitalLibrary.Controllers
{
    /// <summary>
    /// Controlador para la autenticación de usuarios.
    /// </summary>
    [RoutePrefix("api/auth")]
    public class AuthController 

In [33]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "authcontrolador.md"
texto = getattr(res7, "content", None) or str(res7)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\authcontrolador.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\authcontrolador.md
📏 Tamaño (bytes): 8958
🧪 Existe?: True


In [34]:
res8 = ask_with_history("""
 Y a mi codigo  le debo ajustar par que solo si el token esta acceda a todo o puedo hacer acciones 
function registerLoan() {
    const bookId = document.getElementById("loan-book-id").value;
    const userId = document.getElementById("loan-user-id").value;
    const loanDate = document.getElementById("loan-date").value;
    const dueDate = document.getElementById("due-date").value;

    const loanData = {
        bookId: bookId,
        userId: userId,
        loanDate: loanDate,
        dueDate: dueDate,
        isReturned: false // Inicialmente, el préstamo no está devuelto
    };

    fetch('http://localhost:15663/api/loans', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + localStorage.getItem('token') // Si usas JWT, incluye el token
        },
        body: JSON.stringify(loanData)
    })
    .then(response => {
        if (!response.ok) {
            throw new Error('Error al registrar el préstamo');
        }
        return response.json();
    })
    .then(data => {
        alert('Préstamo registrado exitosamente.');
        showLoans(); // Actualizar la lista de préstamos
        document.getElementById("tab-loans").click();
    })
    .catch(error => {
        console.error('Error:', error);
        alert('Error al registrar el préstamo. Por favor, intenta nuevamente.');
        showLoans();
        document.getElementById("tab-loans").click();
    });
}

function editLoanFromForm() {
    const loanId = document.getElementById("edit-loan-id").value;
    editLoan(loanId); // Ahora editLoan solo necesita el ID
}

function editLoan(loanId) {
    const bookId = document.getElementById("edit-loan-book-id").value;
    const userId = document.getElementById("edit-loan-user-id").value;
    const loanDate = document.getElementById("edit-loan-date").value;
    const dueDate = document.getElementById("edit-due-date").value;

    // Validación básica
    if (!bookId || !userId || !loanDate || !dueDate) {
        alert("Por favor completa todos los campos.");
        return;
    }

    const loanData = {
        id: loanId,
        bookId: parseInt(bookId, 10),
        userId: parseInt(userId, 10),
        loanDate: loanDate, // Ya está en formato YYYY-MM-DD (por input type="date")
        dueDate: dueDate
        // No incluimos isReturned para no sobrescribirlo
    };

    fetch(`http://localhost:15663/api/loans/${loanId}`, {
        method: 'PUT',
        headers: {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer ' + localStorage.getItem('token')
        },
        body: JSON.stringify(loanData)
    })
    .then(response => {
        if (!response.ok) {
            return response.text().then(text => {
                throw new Error(`Error ${response.status}: ${text}`);
            });
        }
        return response.json();
    })
    .then(() => {
        alert('Préstamo actualizado exitosamente.');
        showLoans(); // Recargar la lista
        document.getElementById("edit-loan-form").style.display = "none"; // Ocultar form de edición
        document.getElementById("content").style.display = "block"; // Mostrar la lista de nuevo
    })
    .catch(error => {
        console.error('Error:', error);
        alert('Error al actualizar el préstamo: ' + error.message);
    });
}
""")
print(res8)

Para asegurarte de que las acciones de préstamo (como registrar y editar préstamos) solo se realicen si el usuario está autenticado y tiene un token JWT válido, debes implementar una verificación en el lado del servidor y asegurarte de que el cliente maneje adecuadamente la autenticación.

### 1. Asegurando el Controlador de Préstamos

Primero, asegúrate de que tu controlador de préstamos (`LoansController`) esté protegido con el atributo `[Authorize]`. Esto garantiza que solo los usuarios autenticados puedan acceder a los métodos de este controlador.

#### LoansController.cs
```csharp
using System.Web.Http;
using DigitalLibrary.Models;
using DigitalLibrary.Services;

namespace DigitalLibrary.Controllers
{
    [RoutePrefix("api/loans")]
    [Authorize] // Asegurarse de que solo los usuarios autenticados puedan acceder
    public class LoansController : ApiController
    {
        private readonly LoanService _loanService;

        public LoansController()
        {
            _loanSer

In [165]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "analy.md"
texto = getattr(res8, "content", None) or str(res8)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M2\results\analy.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M2\results\analy.md
📏 Tamaño (bytes): 2776
🧪 Existe?: True


In [ ]:
res9 = ask_with_history("""
    Necesito tests unitarios para ASP.NET Framework 4.7.2 usando MSTest (NO xUnit).

Genera tests para mi AuthController que:
1. Usen [TestClass] y [TestMethod]
2. Usen Moq para mocking
3. Usen FluentAssertions 
4. Prueben IHttpActionResult correctamente
5. Mock de HttpRequestMessage y HttpContext

Tests requeridos:
- Register_ReturnsCreated_WhenValid
- Register_ReturnsBadRequest_WhenInvalid
- Login_ReturnsOkWithToken_WhenValid
- Login_ReturnsUnauthorized_WhenInvalid

Usa MSTest, NO xUnit. Usa Assert.AreEqual o FluentAssertions.

""")
print(res9)

Para crear pruebas unitarias para tu `AuthController` utilizando MSTest, Moq y FluentAssertions en un proyecto de ASP.NET Framework 4.7.2, aquí tienes una implementación detallada que cumple con los requisitos que has mencionado. Estas pruebas verificarán el comportamiento de los métodos `Register` y `Login`.

### 1. Configuración del Proyecto de Pruebas

Asegúrate de que tu proyecto de pruebas tenga las siguientes referencias en su archivo `.csproj`:

```xml
<ItemGroup>
  <PackageReference Include="MSTest.TestAdapter" Version="2.2.8" />
  <PackageReference Include="MSTest.TestFramework" Version="2.2.8" />
  <PackageReference Include="Moq" Version="4.16.1" />
  <PackageReference Include="FluentAssertions" Version="5.10.3" />
</ItemGroup>
```

### 2. Implementación de los Tests

A continuación, se presenta la implementación de los tests unitarios para el `AuthController`. Estos tests verificarán el comportamiento de los métodos `Register` y `Login`.

#### AuthControllerTests.cs

```csha

In [45]:
from pathlib import Path

base = Path().resolve()
outdir = base / "results"          # <— plural
outdir.mkdir(parents=True, exist_ok=True)

outfile = outdir / "TEST1.md"
texto = getattr(res9, "content", None) or str(res9)
outfile.write_text(texto, encoding="utf-8")

print("✅ Guardado en:", outfile.resolve())
print("✅ Guardado en:", outfile.resolve())
print("📏 Tamaño (bytes):", outfile.stat().st_size)
print("🧪 Existe?:", outfile.exists())

✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\TEST1.md
✅ Guardado en: C:\Users\00560512\Documents\CURSO IA\M3\results\TEST1.md
📏 Tamaño (bytes): 5939
🧪 Existe?: True
